In [6]:
!pip install trimesh pyvista


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.8/704.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 MB 7.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import trimesh
import pyvista as pv
from noise import pnoise2


In [5]:
class BiomeMapGenerator3D:
    def __init__(self, width=100, height=100, scale=0.1, octaves=6, persistence=0.5, lacunarity=2.0, seed=None):
        self.width = width
        self.height = height
        self.scale = scale
        self.octaves = octaves
        self.persistence = persistence
        self.lacunarity = lacunarity
        self.seed = seed if seed is not None else np.random.randint(0, 100)
        self.biome_heights = {
            "Wasser": -0.5,
            "Sand": 0.0,
            "Grasland": 0.1,
            "Wald": 0.3,
            "Berge": 1.0
        }

    def generate_biome_map(self):
        world = np.zeros((self.height, self.width))

        for y in range(self.height):
            for x in range(self.width):
                noise_value = pnoise2(x * self.scale,
                                      y * self.scale,
                                      octaves=self.octaves,
                                      persistence=self.persistence,
                                      lacunarity=self.lacunarity,
                                      repeatx=self.width,
                                      repeaty=self.height,
                                      base=self.seed)
                world[y][x] = noise_value

        return world

    def get_biome(self, value):
        if value < -0.2:
            return "Wasser"
        elif value < 0:
            return "Sand"
        elif value < 0.2:
            return "Grasland"
        elif value < 0.5:
            return "Wald"
        else:
            return "Berge"

    def generate_height_map(self, biome_map):
        height_map = np.zeros_like(biome_map)

        for y in range(self.height):
            for x in range(self.width):
                biome = self.get_biome(biome_map[y][x])
                height_map[y][x] = self.biome_heights[biome]

        return height_map
    def create_3d_mesh(self, height_map):
        vertices = []
        faces = []

        for y in range(self.height):
            for x in range(self.width):
                z = height_map[y][x]
                vertices.append([x, y, z])

        for y in range(self.height - 1):
            for x in range(self.width - 1):
                v1 = y * self.width + x
                v2 = v1 + 1
                v3 = v1 + self.width
                v4 = v3 + 1
                faces.append([v1, v2, v3])
                faces.append([v2, v4, v3])

        vertices = np.array(vertices)
        faces = np.array(faces)

        mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
        return mesh

    def plot_3d_map(self, mesh):
        plotter = pv.Plotter()
        plotter.add_mesh(mesh, show_edges=True)
        plotter.show()


In [ ]:
# Erstelle ein Objekt der Klasse und generiere die 3D-Karte
generator = BiomeMapGenerator3D(width=100, height=100, scale=0.1, octaves=6, persistence=0.5, lacunarity=2.0, seed=42)
biome_map = generator.generate_biome_map()
height_map = generator.generate_height_map(biome_map)
mesh = generator.create_3d_mesh(height_map)
generator.plot_3d_map(mesh)